In [1]:
from deepface import DeepFace
#liveness
import tensorflow as tf

In [2]:
import cv2, numpy, datetime, pygame, keyboard
import xlwings as xw

pygame 2.5.2 (SDL 2.28.3, Python 3.8.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
model = 'fake_detect.model'
model = tf.keras.models.load_model(model)

In [4]:
#xlwings
#initializing workbook and worksheet
workbook = xw.Book('list.xlsx')
sheet_name = datetime.date.today().isoformat()

try:
    worksheet = workbook.sheets(sheet_name)
except:
    worksheet = workbook.sheets.add(sheet_name)

In [5]:
#xlwings

worksheet.range('A1').value = 'STUDENT'
worksheet.range('B1').value = 'DATE'
worksheet.range('C1').value = 'TIME'

students=[]

In [6]:
s= 2

i = 0

entered = False
al_entered = False

In [7]:
cap = cv2.VideoCapture(0)

#pygame

pygame.init()

pg_txt = (39,112,50)
pg_bg = (154,179,157)

font = cv2.FONT_HERSHEY_SIMPLEX
pygame_font = pygame.font.Font('freesansbold.ttf',32)

screen = pygame.display.set_mode((960,540))

background = pygame.image.load('background_image.jpg')
background = pygame.transform.rotozoom(background,0,0.8)
screen.blit(background,(0,0))


#day today
t0 = datetime.date.today()
t0 = t0.day

running = True

while running:
    
    moment = datetime.datetime.now()
    hour = moment.hour
    minute = moment.minute
    day = moment.day
    month = moment.month
    year = moment.year
    
    date = f"{day}-{month}-{year}"
    time = f"{hour}:{minute}"
        
    screen.blit(background,(0,0))
    
    #check 
    
    if day!=t0:
        t0 = day
        
        worksheet = workbook.sheets.add(date)

        worksheet.range('A1').value = 'STUDENT'
        worksheet.range('B1').value = 'DATE'
        worksheet.range('C1').value = 'TIME'
        students = []
        s=2
        i = 0
        entered = False
        al_entered = False
    
    
    
    
    
    state, frame = cap.read()
    
    if state!=True:
        break
        
    res  = DeepFace.find(frame, db_path='./Database/', enforce_detection=False, model_name='VGG-Face')
    
    if len(res[0]['identity'])>0:
        
        name = res[0]['identity'][0].split('/')[1].split('\\')[1]
        xmin = int(res[0]['source_x'][0])
        ymin = int(res[0]['source_y'][0])
        
        w = res[0]['source_w'][0]
        h = res[0]['source_h'][0]
        
        xmax = int(xmin + w)
        ymax = int(ymin + h)
        
        #liveness
        
        img = frame[ymin:ymax, xmin:xmax]
        img = cv2.resize(img, (32,32))
        img = img.astype('float')/255.0
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = numpy.expand_dims(img, axis=0)
        
        liveness = model.predict(img)
        
        liveness = liveness[0].argmax()
        
        
        

        
        if liveness==1:
            cv2.rectangle(frame, (xmin,ymin),(xmax,ymax), (0,255,0),1)
            cv2.rectangle(frame, (xmin, ymin-25),(xmax, ymin),(255,255,255),-1)
            cv2.putText(frame, name, (xmin,ymin), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),2,cv2.LINE_AA)
            
            n_txt = pygame_font.render(name, True, pg_txt, pg_bg)
            n_txtrect = n_txt.get_rect()
            n_txtrect.center =(960-200, 540//2)
            
            d_txt = pygame_font.render(date, True, pg_txt, pg_bg)
            d_txtrect = d_txt.get_rect()
            d_txtrect.center= (960-200, (540//2)+40)
            
            t_txt = pygame_font.render(time, True, pg_txt, pg_bg)
            t_txtrect = d_txt.get_rect()
            t_txtrect.center = (960-200, (540//2)+80)         
            
            
            screen.blit(n_txt, n_txtrect)
            screen.blit(d_txt, d_txtrect)
            screen.blit(t_txt, t_txtrect)
            
            if keyboard.is_pressed('enter'):
                
                if name not in students:
                    
                    worksheet.range(f'A{s}').value = name
                    worksheet.range(f'B{s}').value = date
                    worksheet.range(f'c{s}').value = time
                    students.append(name)
                    s+=1
                    entered =True
                else:
                    al_entered = True
                    
            if i<30 and entered==True:
                e_txt = pygame_font.render('entered', True, pg_txt, pg_bg)
                e_txtrect = e_txt.get_rect()
                e_txtrect.center = (960-200, (540//2)+120)
                screen.blit(e_txt, e_txtrect)
                i+=1

            elif i<30 and al_entered==True:
                ae_txt = pygame_font.render('already entered', True, pg_txt, pg_bg)
                ae_txtrect = ae_txt.get_rect()
                ae_txtrect.center = (960-200, (540//2)+120)
                screen.blit(ae_txt, e_txtrect)    
                i+=1

            else:
                entered=False
                al_entered = False
                i = 0

    
    
    image = pygame.image.frombuffer(frame.tostring(), frame.shape[1::-1], 'BGR')
    image = pygame.transform.rotozoom(image, 0 , 0.8)
    #round corners--
    size = image.get_size()
    rect_img = pygame.Surface(size, pygame.SRCALPHA)
    pygame.draw.rect(rect_img, (255,255,255), (0,0,*size), border_radius=10)
    
    image = image.copy().convert_alpha()
    image.blit(rect_img, (0,0), None, pygame.BLEND_RGB_MIN)
    #round corner ---
    
    screen.blit(image, (20,80))
    
    
    pygame.display.update()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    #cv2.imshow('attendance', frame)
    
    #c = cv2.waitKey(1)
    
    #if c == ord('q'):
        #break
        
cap.release()
pygame.quit()
#cv2.destroyAllWindows()


There are  9  representations found in  representations_vgg_face.pkl
find function lasts  3.2779107093811035  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.3688926696777344  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.37131357192993164  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.3855469226837158  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.3739297389984131  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.41222333908081055  seconds
1/1 [==============================] - 0s 112ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.2599010467529297  seconds
1/1 [==============================] - 0s 21ms/step
There are  9  representations found in  representations_vgg_face.pk

find function lasts  0.23504114151000977  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.22869300842285156  seconds
1/1 [==============================] - 0s 16ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.4232470989227295  seconds
1/1 [==============================] - 0s 39ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.45022034645080566  seconds
1/1 [==============================] - 0s 36ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.4314699172973633  seconds
1/1 [==============================] - 0s 43ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.44910669326782227  seconds
1/1 [==============================] - 0s 33ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.48299

find function lasts  0.4365665912628174  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.3814995288848877  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.38220643997192383  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.40624141693115234  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.24380064010620117  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.34403228759765625  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.42667508125305176  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.47489261627197266  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.48204708099365234  seconds


find function lasts  0.44084835052490234  seconds
1/1 [==============================] - 0s 32ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.44496870040893555  seconds
1/1 [==============================] - 0s 16ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.24953818321228027  seconds
1/1 [==============================] - 0s 18ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.268695592880249  seconds
1/1 [==============================] - 0s 35ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.32891321182250977  seconds
1/1 [==============================] - 0s 22ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.30805015563964844  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.24729

find function lasts  0.2765696048736572  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.26030683517456055  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.3742642402648926  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.26996445655822754  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.2635319232940674  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.26293444633483887  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.2468576431274414  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.28228330612182617  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.2803816795349121  seconds
The

find function lasts  0.26017284393310547  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.2570624351501465  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.3022630214691162  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.28674888610839844  seconds
1/1 [==============================] - 0s 26ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.2823619842529297  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.2872588634490967  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.2792091369628906  seconds
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.23810529708862305  seconds
There are  9  representations found in  representations_vgg_face.pkl
f